#### What's bigram ?
- Let's say in character level, A character bigram is a pair of consecutive characters in a string of text. For example, in the word "hello",  the bigrams are "he", "el", "ll" and "lo".

#### What's bigram model ?
- A bigram model predicts the next character in a sequence based on the current and previous character. 

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8                                             # block size refers to the small snippets or chunks that we work on
batch_size = 4                                             # batch size refers to the number of chunks that we process in one go
max_iters = 1000
#eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
'''
encoding and decoding, Basically a tokenizer
'''

string_to_int = { ch : i for i,ch in enumerate(chars)}
int_to_string = { i : ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data =  torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:] 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets')
print(y)

inputs:
tensor([[67, 78, 76, 61, 58, 71, 58, 11],
        [72, 68,  1, 54, 67, 60, 71, 78],
        [57, 62, 67, 60,  1, 61, 62, 72],
        [54, 67, 57,  1, 68, 59, 73, 58]], device='cuda:0')
targets
tensor([[78, 76, 61, 58, 71, 58, 11,  3],
        [68,  1, 54, 67, 60, 71, 78,  1],
        [62, 67, 60,  1, 61, 62, 72,  1],
        [67, 57,  1, 68, 59, 73, 58, 67]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
@torch.no_grad()                                     
def estimate_loss():                                  
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)                
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  

    def forward(self, index, targets=None):
        #converting into numbers from token_embedding_table
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            # just keeping the output of shape in B, T, C
            B, T, C = logits.shape                                          # B: batch, T: time, C: channel
            # changing parameters by multiplying            
            logits = logits.view(B*T, C)                                    
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits,loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            #focus only on the last line step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


AA&lECM?P:A()dfy1d"0Qf4P,kb8BN:Y0H,(1CGphSM﻿GlL3]dV4"Fj8T]G63DE_JwMNv5v5b *JkBg;9OZA&8T6 :p75H4Q(WGcg;SCk2﻿nJk3KTn94s,9Z9Oey)_u-3YDZX'l-lmCiFenRbC,J"K4JOtsKSIPrcG0:3UIosPkhBkq8Fa_znntQn)3.AMcxfa_8Wf﻿Q!TJQB.d"Qa&rQsbIkw4v!1F"AM?voHOW;km**9rQ,"5D:.n6fKGD!YjL
6﻿80I,.9﻿iU﻿3-!4sv!WBh-T7:_?q8!h1-f8gT)YA'Fa)_Ni﻿4JvAMcGyl0RnvGBMwfX_tjOT).1NC(.s8b!ucN?_ R
!W0 NisJ[:36&UISQflX__m.0q8J8,&yH?JwvADNtIax6d2pw7OwpZ&gm9B7g124U?OMY1S(M;RSMY'w[I"5h(r;G9J8
!_JvE N﻿3sbM&r3htH478z'Op0kw_﻿9K[hPaqOBV299﻿X8
AAMDZ'(&uYN


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']: .4f}, val loss: {losses['val']: .4f}")
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)                       # To avoid previous gradient affect the current one
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss:  4.9286, val loss:  4.9189
step: 250, train loss:  4.8833, val loss:  4.8706
step: 500, train loss:  4.8054, val loss:  4.8080
step: 750, train loss:  4.7343, val loss:  4.7629
4.634559154510498


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)